In [3]:
#CARGAR LAS BASES DE DATOS
import os
import pandas as pd
import numpy as np
from pandas import DataFrame
os.chdir('C:/Users/Nacha/OneDrive/Desktop/Anaconda/TP4')

In [4]:
# PARTE 1 
# EJERCICIO 2 
data_24 = pd.read_excel('usu_hogar_T124.xlsx')
data_04 = pd.read_stata('Hogar_t104.dta')

In [7]:
# Cambiamos los nombres de las columnas a mayúsculas
data_04.columns = data_04.columns.str.upper()
data_24.columns = data_24.columns.str.upper()

In [29]:
# Combinamos los dataframes
# Dejamos los datos que tengan solo de CABA y GBA
data_04_cleaned = data_04.drop(data_04[~data_04['AGLOMERADO'].isin(['Ciudad de Buenos Aires', 'Partidos del GBA'])].index)
data_04_cleaned = data_04_cleaned.replace({'Ciudad de Buenos Aires': 'CABA', 'Partidos del GBA': 'GBA'})

data_24_cleaned = data_24.drop(data_24[~data_24['AGLOMERADO'].isin([32, 33])].index)
data_24_cleaned = data_24_cleaned.replace({32: 'CABA', 33: 'GBA'})

print("Datos 2004 después de filtrar:", data_04_cleaned.shape[0])
print("Datos 2024 después de filtrar:", data_24_cleaned.shape[0])

# Concatenamos dataframes ya filtrados
data_combined = data_04_cleaned.merge(data_24_cleaned, ignore_index=True)
print('Columnas del dataframe combinado', data_combined.columns)
data_combined.head()

Datos 2004 después de filtrar: 2351
Datos 2024 después de filtrar: 2545
Columnas del dataframe combinado Index(['CODUSU', 'NRO_HOGAR', 'REALIZADA', 'ANO4', 'TRIMESTRE', 'REGION',
       'MAS_500', 'AGLOMERADO', 'PONDERA', 'IV1', 'IV1_ESP', 'IV2', 'IV3',
       'IV3_ESP', 'IV4', 'IV5', 'IV6', 'IV7', 'IV7_ESP', 'IV8', 'IV9', 'IV10',
       'IV11', 'IV12_1', 'IV12_2', 'IV12_3', 'II1', 'II2', 'II3', 'II3_1',
       'II4_1', 'II4_2', 'II4_3', 'II5', 'II5_1', 'II6', 'II6_1', 'II7',
       'II7_ESP', 'II8', 'II8_ESP', 'II9', 'V1', 'V2', 'V21', 'V22', 'V3',
       'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10', 'V11', 'V12', 'V13', 'V14',
       'V15', 'V16', 'V17', 'V18', 'V19_A', 'V19_B', 'IX_TOT', 'IX_MEN10',
       'IX_MAYEQ10', 'ITF', 'DECIFR', 'IDECIFR', 'RDECIFR', 'GDECIFR',
       'PDECIFR', 'ADECIFR', 'IPCF', 'DECCFR', 'IDECCFR', 'RDECCFR', 'GDECCFR',
       'PDECCFR', 'ADECCFR', 'VII1_1', 'VII1_2', 'VII2_1', 'VII2_2', 'VII2_3',
       'VII2_4', 'IDIMPH', 'PONDIH'],
      dtype='object')


C:\Users\Nacha\AppData\Local\Temp\ipykernel_14572\1905505757.py:4: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data_04_cleaned = data_04_cleaned.replace({'Ciudad de Buenos Aires': 'CABA', 'Partidos del GBA': 'GBA'})


,CODUSU,NRO_HOGAR,REALIZADA,ANO4,TRIMESTRE,REGION,MAS_500,AGLOMERADO,PONDERA,IV1,...,PDECCFR,ADECCFR,VII1_1,VII1_2,VII2_1,VII2_2,VII2_3,VII2_4,IDIMPH,PONDIH
0,125098,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,CABA,1299.0,Departamento,...,,04,1.0,0.0,98.0,0.0,0.0,0.0,00000,NaN
1,125397,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,CABA,2018.0,Departamento,...,,00,1.0,0.0,98.0,0.0,0.0,0.0,00000,NaN
2,125515,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,CABA,1486.0,Departamento,...,,05,1.0,0.0,98.0,0.0,0.0,0.0,00010,NaN
3,125558,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,CABA,1674.0,Departamento,...,,06,2.0,0.0,3.0,4.0,0.0,0.0,00000,NaN
4,125599,1.0,Sí,2004.0,1er. Trimestre,Gran Buenos Aires,S,CABA,1471.0,Casa,...,,02,2.0,0.0,1.0,0.0,0.0,0.0,00000,NaN
